In [ ]:
import mne
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.utils import to_categorical

# Load EEG data (adjust path to your files)
def load_eeg_data(file_path):
    raw = mne.io.read_raw_fif(file_path, preload=True)  # Update for your file format
    return raw

# Preprocess EEG (ICA + Bandpass filter)
def preprocess_eeg_data(raw):
    raw.filter(0.5, 45, fir_design='firwin')  # Bandpass filter for the relevant frequency bands
    
    # Fit ICA for artifact removal
    ica = mne.preprocessing.ICA(n_components=20, random_state=42, max_iter=800)
    ica.fit(raw)
    
    # Identify and exclude EOG-related components (for eye artifacts)
    eog_indices, _ = ica.find_bads_eog(raw)
    ica.exclude = eog_indices
    
    raw_cleaned = ica.apply(raw)
    return raw_cleaned

# Feature extraction (spectral features)
def extract_features(raw_cleaned):
    # Apply fixed-length epochs (e.g., 2-second)
    epochs = mne.make_fixed_length_epochs(raw_cleaned, duration=2, overlap=0.5)
    features = epochs.get_data()  # Shape: (n_epochs, n_channels, n_timepoints)
    
    # Use FFT for frequency domain features (Power Spectral Density)
    psd, freqs = mne.time_frequency.psd_welch(epochs, fmin=1, fmax=30)
    psd = np.mean(psd, axis=2)  # Average over time
    
    # Normalize features (important for neural networks)
    scaler = StandardScaler()
    psd = scaler.fit_transform(psd)
    
    return psd

# Load, preprocess, and extract features
raw_data = load_eeg_data('path_to_eeg_data.fif')  # Replace with correct path
raw_cleaned = preprocess_eeg_data(raw_data)
features = extract_features(raw_cleaned)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Bidirectional, BatchNormalization

# Build CNN-LSTM Hybrid Model
def create_cnn_lstm_model(input_shape, output_classes):
    model = Sequential()
    
    # CNN layers for spatial feature extraction
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())  # Normalize activations to stabilize training
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    # Bidirectional LSTM layers for temporal feature extraction
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(0.5))  # Dropout to prevent overfitting
    
    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout to further reduce overfitting
    model.add(Dense(output_classes, activation='softmax'))  # Softmax for multi-class output
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define input shape (number of features per input) and number of output classes
input_shape = (features.shape[1], 1)  # Reshaped input features for CNN
output_classes = len(labels)  # Number of possible output labels

# Create and train the model
model = create_cnn_lstm_model(input_shape, output_classes)
model.summary()

# Reshape data for CNN (add channel dimension)
X_train_cnn = features.reshape((features.shape[0], features.shape[1], 1))

# One-hot encode the labels
labels = ['hello', 'goodbye', 'thanks', 'yes', 'no']  # Example labels
y = np.array([label_to_index[label] for label in labels])  # Convert text labels to numerical index
y_one_hot = to_categorical(y, num_classes=len(labels))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_train_cnn, y_one_hot, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=25, validation_data=(X_test, y_test))


In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
from gtts import gTTS
import os

# Function to convert text to speech
def text_to_speech(text, filename="output.mp3"):
    speech = gTTS(text=text, lang='en', slow=False)
    speech.save(filename)
    os.system(f"start {filename}")  # For Windows, use the appropriate command for your OS

# Predicting a new sample (replace with real-time input if needed)
predicted_index = np.argmax(model.predict(X_test[0:1]), axis=1)[0]  # Example for first test sample
predicted_text = labels[predicted_index]

# Convert predicted text to speech
text_to_speech(predicted_text)
